In [80]:
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import glob
import pandas as pd
import numpy as np
from tensorflow import keras
from collections import Counter

In [81]:
def imageResize(src, max_height):
    
    max_width = np.uint(max_height * 1.5)

    height, width = src.shape[0:2]
    ratio = height / max_height
    
    # 입력 이미지의 높이가 설정한 이미지의 최대 높이보다 크면 리사이즈 수행
    if height > max_height :
        resize_height = max_height
        resize_width = np.uint(width // ratio)
        img = cv2.resize(src, (resize_width, resize_height), cv2.INTER_AREA)
        dy = 0
    
    # 입력 이미지의 높이가 설정한 이미지의 최대 높이보다 작으면 리사이즈 수행하지 않음
    else : 
        resize_height = height
        resize_width = width
        dy = (max_height - resize_height) // 2
        img = src

    dx = (max_width - resize_width) // 2
    mtrx = np.float32([[1,0,dx],[0,1,dy]])


    # 입력 이미지를 규격 프레임의 중심에 넣는다
    return cv2.warpAffine(img, mtrx, (max_width, max_height))

---
# 학습에 사용한 파일 리스트 확인

In [82]:
lenrned_landscape = [file.replace('../Data/Artist/genre/resized/landscape/', '') for file in glob.glob('../Data/Artist/genre/resized/landscape/*.jpg')]
lenrned_landscape[0:5]

['2100.jpg', '50063.jpg', '41344.jpg', '22152.jpg', '95854.jpg']

In [83]:
lenrned_portrait = [file.replace('../Data/Artist/genre/resized/portrait/', '') for file in glob.glob('../Data/Artist/genre/resized/portrait/*.jpg')]
lenrned_portrait[0:5]

['54563.jpg', '88434.jpg', '91824.jpg', '64311.jpg', '27566.jpg']

In [84]:
lenrned_stillLife = [file.replace('../Data/Artist/genre/resized/still life/', '') for file in glob.glob('../Data/Artist/genre/resized/still life/*.jpg')]
lenrned_stillLife[0:5]

['48638.jpg', '47286.jpg', '36696.jpg', '3008.jpg', '64675.jpg']

---
## 학습안한 파일을 각각 200개씩 선택하여 리사이즈

In [85]:
art = pd.read_csv('../Data/Artist/csv/all_select.csv')
art.head()

,Unnamed: 0,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
0,0,Barnett Newman,1955.0,abstract,15530.0,6911.0,9201912.0,wikiart,Color Field Painting,Uriel,train_only,True,102257.jpg
1,1,Barnett Newman,1950.0,abstract,14559.0,6866.0,8867532.0,wikiart,Color Field Painting,Vir Heroicus Sublimis,train_only,True,75232.jpg
2,2,kiri nichol,2013.0,NaN,9003.0,9004.0,1756681.0,NaN,Neoplasticism,NaN,test_only,False,32145.jpg
3,3,kiri nichol,2013.0,NaN,9003.0,9004.0,1942046.0,NaN,Neoplasticism,NaN,test_only,False,20304.jpg
4,4,kiri nichol,2013.0,NaN,9003.0,9004.0,1526212.0,NaN,Neoplasticism,NaN,test_only,False,836.jpg


## 학습하지 않은 풍경화 300장 생성 - 이후 이상한 이미지 삭제

In [86]:
landscape_idx = [idx for idx in art.index if art.iloc[idx]['genre'] == 'landscape']
landscape = art.iloc[landscape_idx].reset_index(drop=True)
landscape.head()

,Unnamed: 0,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
0,11,Wolfgang Paalen,1937.0,landscape,5331.0,8351.0,1407667.0,wikiart,Surrealism,Forbidden Land,train_only,True,70944.jpg
1,13,Utagawa Kunisada,NaN,landscape,9293.0,4688.0,10274760.0,wikiart,Ukiyo-e,Snowy landscape with a woman brandishing a bro...,train_only,True,48138.jpg
2,42,Thomas Gainsborough,1783,landscape,5712.0,4704.0,11791905.0,wikiart,Rococo,The Mall in St. James's Park,train_and_test,True,1164.jpg
3,130,Ferdinand Hodler,1906,landscape,4476.0,4248.0,11568126.0,wikiart,Symbolism,The Grammont,train_and_test,True,20509.jpg
4,136,Rudolf von Alt,1898,landscape,4242.0,4368.0,11514609.0,wikiart,Romanticism,The cemetery in Gastein,train_and_test,True,71952.jpg


In [87]:
# i = 0
# save_path = '../Data/Artist/genre/resized/test_img/landscape/'
# path = '../Data/Artist/original/org_select/'

# for file in landscape['new_filename']:
#     if file not in lenrned_landscape:  # 학습에 사용한 이미지가 아닌경우
        
#         img = cv2.imread(path+file, cv2.IMREAD_COLOR)
#         height, width = img.shape[:2]
#         if height * 1.5 > width :
#             img = imageResize(img, 400)
#             cv2.imwrite(save_path+file, img)
#             i += 1
#     if i > 300:
#         break

## 학습하지 않은 정물화 이미지 300개 생성 - 이상한 이미지 삭제

In [88]:
stillLife_idx = [idx for idx in art.index if art.iloc[idx]['genre'] == 'still life']
stillLife = art.iloc[stillLife_idx].reset_index(drop=True)
stillLife.head()

,Unnamed: 0,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
0,26,Ferdinand Georg Waldmüller,1824,still life,6192.0,4912.0,5842081.0,wikiart,Biedermeier,Fruit Still Life with an Amazon parrot,train_and_test,True,48177.jpg
1,30,Clara Peeters,1612.0,still life,4913.0,6007.0,8684895.0,wikiart,Baroque,Still Life with Flowers and Gold Cups of Honor,train_only,True,28363.jpg
2,169,Juan Gris,1921,still life,5137.0,3396.0,5534860.0,wikipedia,Cubism,The Open Window,train_and_test,True,66723.jpg
3,206,Frans Snyders,NaN,still life,4580.0,3623.0,7779445.0,wikiart,Baroque,"Still Life with dead hares and birds, armchair...",train_and_test,False,80474.jpg
4,340,Mario Tozzi,1922,still life,3442.0,4120.0,3161693.0,wikiart,NaN,Natura Morta,train_and_test,True,13203.jpg


In [89]:
# i = 0
# save_path = '../Data/Artist/genre/resized/test_img/still life/'
# path = '../Data/Artist/original/org_select/'

# for file in stillLife['new_filename']:
#     if file not in lenrned_stillLife:  # 학습에 사용한 이미지가 아닌경우
        
#         img = cv2.imread(path+file, cv2.IMREAD_COLOR)
#         height, width = img.shape[:2]
#         if height * 1.5 > width :
#             img = imageResize(img, 400)
#             cv2.imwrite(save_path+file, img)
#             i += 1
#     if i > 300:
#         break

## 학습하지 않은 초상화 이미지 300개 생성 - 이상한 이미지 삭제

In [90]:
portrait_idx = [idx for idx in art.index if art.iloc[idx]['genre'] == 'portrait']
portrait = art.iloc[portrait_idx].reset_index(drop=True)
portrait.head()

,Unnamed: 0,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
0,48,Jan van Eyck,1433,portrait,4357.0,6021.0,4958316.0,wikiart,Northern Renaissance,A Man in a Turban,train_and_test,False,65895.jpg
1,55,Konstantin Vasilyev,NaN,portrait,4158.0,6119.0,2579399.0,wikiart,Symbolism,Divination,test_only,False,53464.jpg
2,72,Mario Tozzi,1976,portrait,4404.0,5333.0,3809347.0,wikiart,NaN,Volto Misterioso,train_and_test,True,99460.jpg
3,73,Lorenzo Lotto,1547,portrait,5536.0,4226.0,8420464.0,wikiart,High Renaissance,Portrait of Giovanni della Volta with his Wife...,train_and_test,True,5194.jpg
4,79,Paolo Veronese,NaN,portrait,3688.0,6191.0,7445177.0,wikiart,Mannerism (Late Renaissance),Boy with a Greyhound,train_and_test,True,840.jpg


In [91]:
# i = 0
# save_path = '../Data/Artist/genre/resized/test_img/portrait/'
# path = '../Data/Artist/original/org_select/'

# for file in portrait['new_filename']:
#     if file not in lenrned_portrait:  # 학습에 사용한 이미지가 아닌경우
        
#         img = cv2.imread(path+file, cv2.IMREAD_COLOR)
#         height, width = img.shape[:2]
#         if height * 1.5 > width :
#             img = imageResize(img, 400)
#             cv2.imwrite(save_path+file, img)
#             i += 1
#     if i > 300:
#         break


## 장르 맞추기 테스트
### landscape :0, portrait :1, still life :2

In [93]:
# 학습한 모델정보 읽기
model_port_still = keras.models.load_model('../Data/Artist/model/model_v3/port_still_v3.h5')
model_land_still = keras.models.load_model('../Data/Artist/model/model_v3/land_still_v3.h5')
model_land_port = keras.models.load_model('../Data/Artist/model/model_v3/land_port_v3.h5')

In [94]:
classes1 = ['portrait', 'stll life']
classes2= ['landscape', 'stll life']
classes3 = ['landscape', 'portrait']

---
## 풍경화 맞추기

In [95]:
landscape_result = []
for file in glob.glob('../Data/Artist/genre/resized/test_img/landscape/*.jpg'):
    img = np.array(Image.open(file), np.int32)
    img = img.reshape(-1, 400, 600, 3) / 255.0
    pred_1 = model_port_still(img)
    pred_2 = model_land_still(img)
    pred_3 = model_land_port(img)
    pred= [classes1[np.argmax(pred_1)], classes2[np.argmax(pred_2)],classes3[np.argmax(pred_3)]]
    pred_x = Counter(pred)
    landscape_result.append(pred_x.most_common(n=1)[0][0])

In [96]:
Counter(landscape_result)

Counter({'landscape': 47, 'stll life': 5, 'prtrait': 5})

In [97]:
47/(47+5+5)

0.8245614035087719

---
## 정물화 맞추기

In [98]:
stillLife_result = []
for file in glob.glob('../Data/Artist/genre/resized/test_img/still life/*.jpg'):
    img = np.array(Image.open(file), np.int32)
    img = img.reshape(-1, 400, 600, 3) / 255.0
    pred_1 = model_port_still(img)
    pred_2 = model_land_still(img)
    pred_3 = model_land_port(img)
    pred= [classes1[np.argmax(pred_1)], classes2[np.argmax(pred_2)],classes3[np.argmax(pred_3)]]
    pred_x = Counter(pred)
    stillLife_result.append(pred_x.most_common(n=1)[0][0])

In [99]:
Counter(stillLife_result)

Counter({'prtrait': 7, 'stll life': 38, 'landscape': 6})

In [100]:
38/(38+7+6)

0.7450980392156863

---
# 초상화 맞추기

In [101]:
portrait_result = []
for file in glob.glob('../Data/Artist/genre/resized/test_img/portrait/*.jpg'):
    img = np.array(Image.open(file), np.int32)
    img = img.reshape(-1, 400, 600, 3) / 255.0
    pred_1 = model_port_still(img)
    pred_2 = model_land_still(img)
    pred_3 = model_land_port(img)
    pred= [classes1[np.argmax(pred_1)], classes2[np.argmax(pred_2)],classes3[np.argmax(pred_3)]]
    pred_x = Counter(pred)
    portrait_result.append(pred_x.most_common(n=1)[0][0])

In [102]:
Counter(portrait_result)

Counter({'prtrait': 39, 'stll life': 10, 'landscape': 1})

In [103]:
39/(39+10+1)

0.78